In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

In [18]:
eff = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')

In [19]:
pollution_group_dep = pd.read_csv("pollution_group_dep.csv", sep = ",", encoding = 'utf-8', engine = 'python')

In [28]:
pop = pd.read_csv('base_pop_dep.csv', sep = ",", encoding = 'utf-8', engine = 'python')

In [38]:
pop = pop.drop(['Unnamed: 0'], axis=1)

In [43]:
eff

,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,ntop,npop,prev,niveau_prioritaire,libelle_classe_age,libelle_sexe,tri
0,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,1.0,971,290.0,4210.0,6.912,"2,3",de 80 à 84 ans,hommes,51.0
1,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,3.0,973,40.0,650.0,5.864,"2,3",de 80 à 84 ans,hommes,51.0
2,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,4.0,999,410.0,5630.0,7.340,"2,3",de 80 à 84 ans,hommes,51.0
3,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,6.0,999,20.0,350.0,4.913,"2,3",de 80 à 84 ans,hommes,51.0
4,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,11.0,75,2960.0,20820.0,14.221,"2,3",de 80 à 84 ans,hommes,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2036148,2015,Cancers,Cancer de la prostate,NaN,sup_CanPros_cat,45-49,1.0,44.0,55,NaN,5390.0,NaN,2,de 45 à 49 ans,hommes,20.0
2036149,2015,Cancers,Cancer de la prostate,NaN,sup_CanPros_cat,45-49,1.0,44.0,88,NaN,11930.0,NaN,2,de 45 à 49 ans,hommes,20.0
2036150,2015,Cancers,Cancer de la prostate,NaN,sup_CanPros_cat,45-49,1.0,53.0,22,NaN,18390.0,NaN,2,de 45 à 49 ans,hommes,20.0
2036151,2015,Cancers,Cancer de la prostate,NaN,sup_CanPros_cat,45-49,1.0,75.0,19,NaN,7380.0,NaN,2,de 45 à 49 ans,hommes,20.0


In [40]:
# Fusion des tables effectifs, pop et pollution_group_dep

merged_df = pd.merge(eff, pollution_group_dep, left_on=['dept'], right_on=['DEP'], 
                    how='inner')
pop_eff_pol = pd.merge(merged_df, pop, left_on=['DEP'], right_on=['numero_dep'], 
                    how='inner')

# Supprimer les colonnes redondantes
pop_eff_pol = pop_eff_pol.drop(['numero_dep', 'dept'], axis=1)

In [41]:
pop_eff_pol

,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,ntop,npop,...,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,Routier,Tertiaire,nom_dep,total_habitants
0,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,11.0,2960.0,20820.0,...,183.766008,1.186577e+06,27358.781206,147965.117571,434314.469384,1.934580e+06,1.625583e+06,1.331630e+06,Paris,2102650
1,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,9.0,11.0,7990.0,50520.0,...,183.766008,1.186577e+06,27358.781206,147965.117571,434314.469384,1.934580e+06,1.625583e+06,1.331630e+06,Paris,2102650
2,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,90-94,2.0,11.0,1450.0,14960.0,...,183.766008,1.186577e+06,27358.781206,147965.117571,434314.469384,1.934580e+06,1.625583e+06,1.331630e+06,Paris,2102650
3,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,95et+,2.0,11.0,370.0,7540.0,...,183.766008,1.186577e+06,27358.781206,147965.117571,434314.469384,1.934580e+06,1.625583e+06,1.331630e+06,Paris,2102650
4,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,tsage,2.0,11.0,41370.0,1119660.0,...,183.766008,1.186577e+06,27358.781206,147965.117571,434314.469384,1.934580e+06,1.625583e+06,1.331630e+06,Paris,2102650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629098,2015,Cancers,Cancer de la prostate,NaN,sup_CanPros_cat,10-14,1.0,53.0,NaN,17820.0,...,293.809962,5.929728e+05,135543.139833,65762.797711,283874.799468,4.761495e+05,1.273879e+06,3.598942e+05,Côtes-d'Armor,607834
1629099,2015,Cancers,Cancer de la prostate,NaN,sup_CanPros_cat,15-19,1.0,53.0,NaN,16990.0,...,293.809962,5.929728e+05,135543.139833,65762.797711,283874.799468,4.761495e+05,1.273879e+06,3.598942e+05,Côtes-d'Armor,607834
1629100,2015,Cancers,Cancer de la prostate,NaN,sup_CanPros_cat,15-19,9.0,53.0,NaN,33590.0,...,293.809962,5.929728e+05,135543.139833,65762.797711,283874.799468,4.761495e+05,1.273879e+06,3.598942e+05,Côtes-d'Armor,607834
1629101,2015,Cancers,Cancer de la prostate,NaN,sup_CanPros_cat,40-44,1.0,53.0,NaN,17780.0,...,293.809962,5.929728e+05,135543.139833,65762.797711,283874.799468,4.761495e+05,1.273879e+06,3.598942e+05,Côtes-d'Armor,607834


In [23]:
# Fusion des tables effectifs, pop et pollution_group_dep
merged_df = pd.merge(eff, pollution_group_dep, left_on='dept', right_on='DEP', how='inner')
pop_eff_pol = pd.merge(merged_df, pop, left_on='DEP', right_on='numero_dep', how='inner')

# Supprimer les colonnes redondantes
pop_eff_pol = pop_eff_pol.drop(['dept', 'DEP', 'numero_dep'], axis=1)

# Renommer la colonne finale
pop_eff_pol = pop_eff_pol.rename(columns={'pop_col': 'population'})

# Afficher le DataFrame résultant
pop_eff_pol

,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,ntop,npop,...,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,Routier,Tertiaire,Unnamed: 0,dep,total_habitants
0,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,1.0,11.0,2960.0,20820.0,...,1.186577e+06,27358.781206,147965.117571,434314.469384,1.934580e+06,1.625583e+06,1.331630e+06,75,Paris,2102650
1,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,80-84,9.0,11.0,7990.0,50520.0,...,1.186577e+06,27358.781206,147965.117571,434314.469384,1.934580e+06,1.625583e+06,1.331630e+06,75,Paris,2102650
2,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,90-94,2.0,11.0,1450.0,14960.0,...,1.186577e+06,27358.781206,147965.117571,434314.469384,1.934580e+06,1.625583e+06,1.331630e+06,75,Paris,2102650
3,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,95et+,2.0,11.0,370.0,7540.0,...,1.186577e+06,27358.781206,147965.117571,434314.469384,1.934580e+06,1.625583e+06,1.331630e+06,75,Paris,2102650
4,2020,Traitements du risque vasculaire (hors patholo...,Traitements hypolipémiants (hors pathologies),Traitements hypolipémiants (hors pathologies),top_FHypoLi_med,tsage,2.0,11.0,41370.0,1119660.0,...,1.186577e+06,27358.781206,147965.117571,434314.469384,1.934580e+06,1.625583e+06,1.331630e+06,75,Paris,2102650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629098,2015,Cancers,Cancer de la prostate,NaN,sup_CanPros_cat,10-14,1.0,53.0,NaN,17820.0,...,5.929728e+05,135543.139833,65762.797711,283874.799468,4.761495e+05,1.273879e+06,3.598942e+05,22,Côtes-d'Armor,607834
1629099,2015,Cancers,Cancer de la prostate,NaN,sup_CanPros_cat,15-19,1.0,53.0,NaN,16990.0,...,5.929728e+05,135543.139833,65762.797711,283874.799468,4.761495e+05,1.273879e+06,3.598942e+05,22,Côtes-d'Armor,607834
1629100,2015,Cancers,Cancer de la prostate,NaN,sup_CanPros_cat,15-19,9.0,53.0,NaN,33590.0,...,5.929728e+05,135543.139833,65762.797711,283874.799468,4.761495e+05,1.273879e+06,3.598942e+05,22,Côtes-d'Armor,607834
1629101,2015,Cancers,Cancer de la prostate,NaN,sup_CanPros_cat,40-44,1.0,53.0,NaN,17780.0,...,5.929728e+05,135543.139833,65762.797711,283874.799468,4.761495e+05,1.273879e+06,3.598942e+05,22,Côtes-d'Armor,607834


In [ ]:
#Les données datent de 2015 à 2020, 
#on prend le classement par PIB par habitant par département et on garde 
#le dep avec le PIB par habitant le plus élevé (Paris (75), pib/habitant de 127 132€)
#celui où il est le plus faible (la Creuse (23), pib/habitant de 23 192€),
#et celui qui est le plus près de la moyenne nationale (Bas-Rhin (67)).
#Le PIB moyen par habitant en 2021 est de 37 431€ par an.
# source : wikipedia

In [ ]:
#idée : prendre en compte la variable niveau_prioritaire pour comparer